In [1]:
#import required python libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#libraries for NLP
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [3]:
import nltk

# Download the stopwords dataset for English
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rahul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rahul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rahul\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\rahul\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [4]:
sw=stopwords.words('english')
lm=WordNetLemmatizer()

In [6]:
#import the dataset
df=pd.read_csv("spam.csv")
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [7]:
df=df[["v1","v2"]]

In [8]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [9]:
df.rename(columns={'v1':'category', 'v2': 'message'}, inplace=True)
df.head()

,category,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [10]:
df.columns

Index(['category', 'message'], dtype='object')

In [11]:
df.shape

(5572, 2)

In [12]:
df.dtypes

category    object
message     object
dtype: object

In [13]:
df.isnull().sum()

category    0
message     0
dtype: int64

In [14]:
#handle duplicates
df.duplicated().sum()

403

In [15]:
df.drop_duplicates(inplace=True)

In [16]:
df.shape

(5169, 2)

In [17]:
#Text preprocessing
import regex

In [18]:
df['message'][:5] #accessing first 5 rows
msg=[]
for i in df['message']:
    t=regex.sub('[^A-Za-z0-9]','',i)
    t=t.lower()
    t=word_tokenize(t)
    t=[i for i in t if i not in sw]
    t=[lm.lemmatize(i) for i in t]
    t=" ".join(t)
    msg.append(t)
print(msg)   

['gountiljurongpointcrazyavailableonlyinbugisngreatworldlaebuffetcinetheregotamorewat', 'oklarjokingwifuoni', 'freeentryin2awklycomptowinfacupfinaltkts21stmay2005textfato87121toreceiveentryquestionstdtxtratetcsapply08452810075over18s', 'udunsaysoearlyhorucalreadythensay', 'nahidontthinkhegoestousfhelivesaroundherethough', 'freemsgheytheredarlingitsbeen3weeksnowandnowordbackidlikesomefunyouupforitstilltbokxxxstdchgstosend150torcv', 'evenmybrotherisnotliketospeakwithmetheytreatmelikeaidspatent', 'asperyourrequestmellemelleoruminnaminungintenurunguvettamhasbeensetasyourcallertuneforallcallerspress9tocopyyourfriendscallertune', 'winnerasavaluednetworkcustomeryouhavebeenselectedtoreceivea900prizerewardtoclaimcall09061701461claimcodekl341valid12hoursonly', 'hadyourmobile11monthsormoreurentitledtoupdatetothelatestcolourmobileswithcameraforfreecallthemobileupdatecofreeon08002986030', 'imgonnabehomesoonandidontwanttotalkaboutthisstuffanymoretonightkivecriedenoughtoday', 'sixchancestowincashfrom

In [20]:
x=msg
y=df['category']
print(type(x))
print(type(y))

<class 'list'>
<class 'pandas.core.series.Series'>


In [21]:
from sklearn.feature_extraction.text import CountVectorizer

In [22]:
cv=CountVectorizer(max_features=1500)
x_cv=cv.fit_transform(x).toarray()
print(x_cv.shape)

(5169, 1500)


In [23]:
from sklearn.model_selection import train_test_split

In [24]:
x_train,x_test,y_train,y_test=train_test_split(x_cv,y,test_size=0.25)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(3876, 1500)
(1293, 1500)
(3876,)
(1293,)


In [25]:
#building the ML model
from sklearn.naive_bayes import MultinomialNB

In [26]:
mnb=MultinomialNB()
mnb.fit(x_train,y_train)

MultinomialNB()

In [27]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [28]:
def eval_model(y_test,y_pred):
    cm=confusion_matrix(y_test,y_pred)
    print(cm)
    print("Accuracy Score:",accuracy_score(y_test,y_pred))
    print("Classification report:",classification_report(y_test,y_pred))

In [29]:
def model_score(model):
    print("Training score :",model.score(x_train,y_train))
    print("Testing score :",model.score(x_test,y_test))

In [30]:
y_pred_mnb=mnb.predict(x_test)

In [31]:
model_score(mnb)

Training score : 0.8753869969040248
Testing score : 0.868522815158546


In [32]:
eval_model(y_test,y_pred_mnb)

[[1123    0]
 [ 170    0]]
Accuracy Score: 0.868522815158546
Classification report:               precision    recall  f1-score   support

         ham       0.87      1.00      0.93      1123
        spam       0.00      0.00      0.00       170

    accuracy                           0.87      1293
   macro avg       0.43      0.50      0.46      1293
weighted avg       0.75      0.87      0.81      1293



C:\Users\rahul\anac\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rahul\anac\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rahul\anac\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
print(y_pred_mnb)

['ham' 'ham' 'ham' ... 'ham' 'ham' 'ham']
